# ディープラーニングによる自然言語処理: 固有表現認識モデルの実装

## 環境のセットアップ

本ノートブックの実行に必要なパッケージをインストールします。

In [ ]:
# AllenNLPをインストール
!pip install allennlp==2.9.2 allennlp-models==2.9.0 google-cloud-storage==2.1.0
# fugashiをUniDicの依存ライブラリを含めてインストール
!pip install fugashi[unidic]
# UniDicの辞書ファイルをダウンロード
!python -m unidic download
# PyKNPとSpaCyをインストール
!pip install pyknp==0.5.0 spacy==3.0.6

     |████████████████████████████████| 719 kB 5.2 MB/s 
     |████████████████████████████████| 463 kB 52.0 MB/s 
     |████████████████████████████████| 106 kB 95.5 MB/s 
     |████████████████████████████████| 3.8 MB 54.1 MB/s 
     |████████████████████████████████| 1.2 MB 30.6 MB/s 
     |████████████████████████████████| 248 kB 29.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 592 kB 63.8 MB/s 
     |████████████████████████████████| 1.8 MB 52.6 MB/s 
     |████████████████████████████████| 125 kB 74.5 MB/s 
     |████████████████████████████████| 77 kB 9.5 MB/s 
     |████████████████████████████████| 325 kB 76.6 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
     |████████████████████████████████| 56 kB 6.3 MB/s 
     |████████████████████████████████| 1.5 MB 49.0 MB/s 
     |████████████████████

     |████████████████████████████████| 568 kB 5.1 MB/s 
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7426 sha256=6def872292c8da9f8757f1d82dee63b928b2b965c1f97beac153938a0ff92ff0
  Stored in directory: /root/.cache/pip/wheels/ce/4d/f1/170bb74b559ca338113c0315c9805e16dfd0a12411ec6b1122
Successfully built unidic
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:28<00:00, 18.4MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.7/dist-packages/unidic/dicdir
     |████████████████████████████████| 42 kB 635 kB/s 
     |████████████████████████████████| 12.8 MB 8.1 MB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 451 kB 60.2 MB/s 
     |████████████████████████████████| 653 kB 29.9 MB/s 
     |████████████████████████████████| 9.1 MB 51.1 MB/s 
  Attempting uninstall: cata

## データセットのセットアップ

### データセットのダウンロード

In [ ]:
!mkdir -p data/kwdlc
!git clone https://github.com/ku-nlp/KWDLC.git data/kwdlc/repo

Cloning into 'data/kwdlc/repo'...
remote: Enumerating objects: 22523, done.
remote: Counting objects: 100% (22523/22523), done.
remote: Compressing objects: 100% (6736/6736), done.
remote: Total 22523 (delta 15841), reused 22461 (delta 15783), pack-reused 0
Receiving objects: 100% (22523/22523), 16.30 MiB | 19.73 MiB/s, done.
Resolving deltas: 100% (15841/15841), done.


### データセットをCoNLL-2003形式に変換

In [ ]:
import glob
import random
import re
from pyknp import BList

def add_ne_tag_to_mrphs(sentence):
    """基本句に付与されている固有表現ラベルを各形態素に付与"""
    # 文（sentence）に含まれる基本句（tag）を順に処理
    for tag in sentence.tag_list():
        # 基本句に<NE:LOCATION:新宿区役所>のような形式で付与されている固有表現ラベルを
        # 正規表現を使って抜き出す
        match = re.search(r"<NE:(.+?):(.+?)>", tag.fstring)

        # 固有表現ラベルがなかった場合は飛ばす
        if not match:
            continue

        # 固有表現の型（例:LOCATION）と文字列（新宿区役所）を
        # それぞれne_type, ne_textに格納
        ne_type, ne_text = match.groups()

        # 曖昧性が高いなどの理由によりラベル付けが困難なものにはOPTIONALラベルが
        # 付与されており、このラベルは対象としない
        if ne_type == "OPTIONAL":
            continue

        # 基本句に含まれる形態素を逆順に処理
        for mrph in reversed(tag.mrph_list()):
            # 固有表現末尾の形態素を探す
            if not ne_text.endswith(mrph.midasi):
                continue

            # 固有表現の末尾の形態素から逆順に文中の形態素を辿っていき
            # 固有表現に含まれる全ての形態素のID列を得る
            ne_mrph_ids = []
            ne_string = ""
            for i in range(mrph.mrph_id, -1, -1):
                ne_mrph_ids.insert(0, i)
                ne_string = sentence.mrph_list()[i].midasi + ne_string
                if ne_string == ne_text:
                    break

            # 各形態素に固有表現ラベルを付与
            for i, ne_mrph_id in enumerate(ne_mrph_ids):
                target_mrph = sentence.mrph_list()[ne_mrph_id]
                # 固有表現の先頭はラベルB、それ以外はラベルI
                target_mrph.fstring += "<NE:{}:{}/>".format(
                    ne_type, "B" if i == 0 else "I")

def write_file(out_file, documents):
    """データセットをファイルに書き出す"""
    with open(out_file, "w") as f:
        for document in documents:
            for sentence in document:
                for mrph in sentence.mrph_list():
                    match = re.search(r"<NE:(.+?):([BI])/>", mrph.fstring)
                    if match:
                        # B-PERSONのような形式の固有表現ラベルを作成
                        ne_tag = "{}-{}".format(match.group(2), match.group(1))
                    else:
                        # 固有表現ラベルが無い場合は"O"ラベルを付与
                        ne_tag = "O"
                    # 1カラム目に単語、4カラム目に固有表現ラベルを書く
                    # それ以外のカラムは利用しない
                    f.write("{} N/A N/A {}\n".format(mrph.midasi, ne_tag))
                f.write("\n")

documents = []
# データセットに含まれる各ファイルを順に読み込む
for doc_file in sorted(glob.glob("data/kwdlc/repo/knp/*/*", recursive=True)):
    sentences = []
    with open(doc_file) as f:
        # ファイルに含まれる文とその固有表現ラベルを読み込む
        buf = ""
        for line in f:
            buf += line
            # EOSは文末を示す
            if "EOS" in line:
                sentence = BList(buf)
                add_ne_tag_to_mrphs(sentence)
                sentences.append(sentence)
                buf = ""
    documents.append(sentences)

# データセットをランダムに並べ替える
random.seed(1234)
random.shuffle(documents)

# データセットの80%を訓練データ、10%を検証データ、10%をテストデータとして用いる
num_train = int(0.8 * len(documents))
num_test = int(0.1 * len(documents))
train_documents = documents[:num_train]
validation_documents = documents[num_train:-num_test]
test_documents = documents[-num_test:]

# データセットをファイルに書き込む
write_file("data/kwdlc/kwdlc_ner_train.txt", train_documents)
write_file("data/kwdlc/kwdlc_ner_validation.txt", validation_documents)
write_file("data/kwdlc/kwdlc_ner_test.txt", test_documents)

In [ ]:
!ls data/kwdlc/*.txt

data/kwdlc/kwdlc_ner_test.txt	data/kwdlc/kwdlc_ner_validation.txt
data/kwdlc/kwdlc_ner_train.txt


In [ ]:
!head -n5 data/kwdlc/kwdlc_ner_train.txt

自然 N/A N/A O
豊かな N/A N/A O
この N/A N/A O
場所 N/A N/A O
で N/A N/A O


## モデルの実装

In [ ]:
model_config = """{
    "random_seed": 1,
    "pytorch_seed": 1,
    "train_data_path": "data/kwdlc/kwdlc_ner_train.txt",
    "validation_data_path": "data/kwdlc/kwdlc_ner_validation.txt",
    "dataset_reader": {
        "type": "conll2003",
        "tag_label": "ner",
        "token_indexers": {
            "tokens": {
                "type": "single_id"
            }
        }
    },
    "data_loader": {
        "batch_size": 32,
        "shuffle": true
    },
    "validation_data_loader": {
        "batch_size": 32,
        "shuffle": false
    },
    "vocabulary": {},
    "datasets_for_vocab_creation": ["train"],
    "model": {
        "type": "crf_tagger",
        "label_encoding": "BIO",
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "type": "embedding",
                    "embedding_dim": 300,
                    "pretrained_file": "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.vec.gz"
                }
            }
        },
        "encoder": {
           "type": "lstm",
           "input_size": 300,
           "hidden_size": 32,
           "bidirectional": true
        }
    },
    "trainer": {
        "cuda_device": 0,
        "validation_metric": "+f1-measure-overall",
        "optimizer": {
            "type": "adam"
        },
        "num_epochs": 10,
        "patience": 3,
        "callbacks": [
            {
                "type": "tensorboard"
            }
        ]
    }
}"""
with open("kwdlc_ner.jsonnet", "w") as f:
  f.write(model_config)

## モデルの訓練

In [ ]:
# 出力ディレクトリが既にあった場合は削除
!rm -rf exp_kwdlc_ner
# 訓練を実行
!allennlp train --serialization-dir exp_kwdlc_ner kwdlc_ner.jsonnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
2022-04-10 14:49:37,348 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-04-10 14:49:37,384 - INFO - allennlp.common.params - evaluation = None
2022-04-10 14:49:37,384 - INFO - allennlp.common.params - include_in_archive = None
2022-04-10 14:49:37,385 - INFO - allennlp.common.params - random_seed = 1
2022-04-10 14:49:37,385 - INFO - allennlp.common.params - numpy_seed = 1337
2022-04-10 14:49:37,385 - INFO - allennlp.common.params - pytorch_seed = 1
2022-04-10 14:49:37,387 - INFO - allennlp.common.checks - Pytorch version: 1.10.0+cu111
2022-04-10 14:49:37,388 - INFO - allennlp.common.params - type = default
2022-04-10 14:49:37,388 - INFO - allennlp.common.params - dataset_reader.type = conll2003
2022-04-10 14:49:37,389 - INFO - allennlp.common.params - d

## 性能の評価

In [ ]:
!allennlp evaluate exp_kwdlc_ner/model.tar.gz data/kwdlc/kwdlc_ner_test.txt

2022-04-10 14:56:55,586 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-04-10 14:56:55,591 - INFO - allennlp.models.archival - loading archive file exp_kwdlc_ner/model.tar.gz
2022-04-10 14:56:55,591 - INFO - allennlp.models.archival - extracting archive file exp_kwdlc_ner/model.tar.gz to temp dir /tmp/tmpm269v_te
2022-04-10 14:56:55,833 - INFO - allennlp.data.vocabulary - Loading token dictionary from /tmp/tmpm269v_te/vocabulary.
2022-04-10 14:56:55,926 - INFO - allennlp.models.archival - removing temporary unarchived model dir at /tmp/tmpm269v_te
2022-04-10 14:56:55,931 - INFO - allennlp.common.checks - Pytorch version: 1.10.0+cu111
2022-04-10 14:56:55,931 - INFO - allennlp.commands.evaluate - Reading evaluation data from kwdlc_ner_test
loading instances: 0it [00:00, ?it/s]2022-04-10 14:56:55,933 - INFO - allennlp.data.dataset_readers.conll2003 - Reading instances from lines in file at: data/kwdlc/kwdlc_ner_test.txt
loading instances: 1607it [00:00, 20057.21it

## 出力の視覚化

In [ ]:
!allennlp predict --output-file exp_kwdlc_ner/validation_predictions.json --silent --use-dataset-reader exp_kwdlc_ner/model.tar.gz data/kwdlc/kwdlc_ner_validation.txt

2022-04-10 14:57:04,564 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-04-10 14:57:04,569 - INFO - allennlp.models.archival - loading archive file exp_kwdlc_ner/model.tar.gz
2022-04-10 14:57:04,569 - INFO - allennlp.models.archival - extracting archive file exp_kwdlc_ner/model.tar.gz to temp dir /tmp/tmpc2f6o8ky
2022-04-10 14:57:04,810 - INFO - allennlp.common.params - dataset_reader.type = conll2003
2022-04-10 14:57:04,810 - INFO - allennlp.common.params - dataset_reader.max_instances = None
2022-04-10 14:57:04,810 - INFO - allennlp.common.params - dataset_reader.manual_distributed_sharding = False
2022-04-10 14:57:04,811 - INFO - allennlp.common.params - dataset_reader.manual_multiprocess_sharding = False
2022-04-10 14:57:04,811 - INFO - allennlp.common.params - dataset_reader.token_indexers.tokens.type = single_id
2022-04-10 14:57:04,811 - INFO - allennlp.common.params - dataset_reader.token_indexers.tokens.namespace = tokens
2022-04-10 14:57:04,811 - INFO 

In [ ]:
import json
from allennlp.data.dataset_readers.dataset_utils.span_utils import bio_tags_to_spans
from spacy import displacy
from spacy.tokens import Doc, Span
from spacy.vocab import Vocab

def create_doc_instance(words, labels):
    """単語のリストとラベルのリストからエンティティの情報を含んだDocインスタンスを作成"""
    # 単語のリストからDocインスタンスを作成
    # ラベルのリストもDocインスタンスに紐付ける
    doc = Doc(Vocab(), words=words, user_data={"labels": labels})

    # ラベルのリストをエンティティの型名と範囲を含んだタプルのリストに変換
    entities = bio_tags_to_spans(labels)

    spans = []
    # エンティティの型名と範囲のリストを個別に処理し、Spanインスタンスのリストを作成
    for entity_type, (start, end) in entities:
        # エンティティの開始・終了位置、型名を使ってSpanインスタンスを作成
        # 終了位置としてbio_tags_to_spans関数はエンティティの範囲内の最後の単語の位置を返すが、
        # Spanクラスにはエンティティの最後の単語の次の単語の位置を指定する必要があるため、
        # end + 1とする
        span = Span(doc, start, end + 1, entity_type)
        spans.append(span)
    # DocインスタンスにSpanインスタンスのリストを紐付ける
    doc.set_ents(spans)
    return doc

# モデルの予測結果を読み込む
output_docs = []
with open("exp_kwdlc_ner/validation_predictions.json") as output_file:
    for line in output_file:
        result = json.loads(line)
        doc = create_doc_instance(result["words"], result["tags"])
        output_docs.append(doc)

# データセットから正解データを読み込む
gold_docs = []
with open("data/kwdlc/kwdlc_ner_validation.txt") as gold_file:
    words, labels = [], []
    for line in gold_file:
        line = line.rstrip("\n")
        if line:
            # 単語（1カラム目）、 固有表現ラベル（4カラム目）以外は利用しない
            word, _, _, label = line.split(" ")
            words.append(word)
            labels.append(label)
        # 空行が文の切れ目
        else:
            doc = create_doc_instance(words, labels)
            gold_docs.append(doc)
            words, labels = [], []

In [ ]:
import warnings

# 与えられた文にエンティティがない場合にdisplaCyが表示する警告を無効化
warnings.simplefilter("ignore")

# エンティティの型に設定する色
ENTITY_COLORS = {
    "ARTIFACT": "#55efc4",
    "DATE": "#81ecec",
    "LOCATION": "#74b9ff",
    "MONEY": "#a29bfe",
    "ORGANIZATION": "#dfe6e9",
    "PERCENT": "#ffeaa7",
    "PERSON": "#fab1a0",
    "TIME": "#fd79a8",
}

# 検証データセットの最初から300件の文を順に処理
for output_doc, gold_doc in zip(output_docs[:300], gold_docs[:300]):
    # モデルの予測と正解データのどちらにもエンティティがない場合は飛ばす
    if not output_doc.ents and not gold_doc.ents:
        continue

    # モデルの予測と正解データのラベル列が一致していない場合は双方を表示する
    if output_doc.user_data["labels"] != gold_doc.user_data["labels"]:
        print("モデルの予測：")
        displacy.render(output_doc, style="ent", jupyter=True, options={"colors": ENTITY_COLORS})
        print("正解：")
        displacy.render(gold_doc, style="ent", jupyter=True, options={"colors": ENTITY_COLORS})
    else:
        displacy.render(output_doc, style="ent", jupyter=True, options={"colors": ENTITY_COLORS})
    print("----------")

----------


----------
モデルの予測：


正解：


----------
モデルの予測：


正解：


...

正解：


----------
モデルの予測：


正解：


----------


----------
